# Введение в нейронные сети
## Классификация MNIST

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm

### Подготовка данных для использования
 - Получение Датасета
 - Получение Даталоадера
 - Просмотр данных

In [ ]:
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=ToTensor())
test_dataset = datasets.MNIST(root="./data", train=False, transform=ToTensor())

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=4)

In [ ]:
### Целевое значение
next(iter(train_dataloader))[1]

In [ ]:
### Torch tensor
next(iter(train_dataloader))[0].shape

In [ ]:
for_show = next(iter(train_dataloader))[0]
plt.imshow(for_show.numpy()[0][0])

### Построим нейронную сеть

In [ ]:
class MyNN(torch.nn.Module):
    def __init__(self, dims=[256, 64, 32], dropout_p=0):
        super().__init__()
        self.Linear1 = torch.nn.Linear(28 * 28, dims[0])
        self.Linear2 = torch.nn.Linear(dims[0], dims[1])
        self.Linear3 = torch.nn.Linear(dims[1], dims[2])
        self.Linear4 = torch.nn.Linear(dims[2], 10)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(dropout_p)
        self.SoftMax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = x.squeeze(1)
        x = x.flatten(1)
        x = self.dropout(x)
        x = self.Linear1(x)
        x = self.relu(x)
        x = self.Linear2(x)
        x = self.relu(x)
        x = self.Linear3(x)
        x = self.relu(x)
        x = self.Linear4(x)
        x = self.SoftMax(x)
        return x

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = MyNN()

### Функция, вычисляющая лосс

In [ ]:
def loss_function(true, pred):
    loss = torch.nn.CrossEntropyLoss()
    return loss(pred, true)

### Оптимизатор

In [ ]:
optimizer = torch.optim.SGD(params=model.parameters(), lr=1e-3)

### Построим процесс обучения нейронной сети

In [ ]:
NUM_EPOCHS = 250

In [ ]:
train_loss = []
test_loss = []
for epoch in tqdm(range(NUM_EPOCHS)):
    model.train(True)
    train_loss_epoch = 0
    test_loss_epoch = 0
    for X, y in train_dataloader:
        true = y
        model.zero_grad()
        predictions = model.forward(X)
        loss_value = loss_function(true, predictions)
        loss_value.backward()
        train_loss_epoch += loss_value.item() / len(train_dataloader)
        optimizer.step()
    train_loss.append(train_loss_epoch)
    model.train(False)
    for X, y in test_dataloader:
        true = y
        predictions = model.forward(X)
        loss_value = loss_function(true, predictions)
        test_loss_epoch += loss_value.item() / len(test_dataloader)
    test_loss.append(test_loss_epoch)
    # print(epoch, train_loss[-1], test_loss[-1])

In [ ]:
model(X).shape

In [ ]:
sns.lineplot(train_loss)
sns.lineplot(test_loss)

In [ ]:
preds = []
reals = []
for X, y in test_dataloader:
    preds.append(model(X).detach().numpy())
    reals.append(y.numpy())

In [ ]:
preds = (np.concatenate(preds).argmax(axis=1)).astype(int)

In [ ]:
reals = np.concatenate(reals)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [ ]:
print(classification_report(reals, preds))

In [ ]:
model2 = MyNN(dropout_p=0.3)
optimizer2 = torch.optim.SGD(params=model2.parameters(), lr=1e-3)
train_loss = []
test_loss = []
for epoch in tqdm(range(NUM_EPOCHS)):
    model2.train(True)
    train_loss_epoch = 0
    test_loss_epoch = 0
    for X, y in train_dataloader:
        true = y
        model2.zero_grad()
        predictions = model2.forward(X)
        loss_value = loss_function(true, predictions)
        loss_value.backward()
        train_loss_epoch += loss_value.item() / len(train_dataloader)
        optimizer2.step()
    train_loss.append(train_loss_epoch)
    model2.train(False)
    for X, y in test_dataloader:
        true = y
        predictions = model2.forward(X)
        loss_value = loss_function(true, predictions)
        test_loss_epoch += loss_value.item() / len(test_dataloader)
    test_loss.append(test_loss_epoch)
    # print(epoch, train_loss[-1], test_loss[-1])

preds = []
reals = []
for X, y in test_dataloader:
    preds.append(model2(X).detach().numpy())
    reals.append(y.numpy())
preds = (np.concatenate(preds).argmax(axis=1)).astype(int)
reals = np.concatenate(reals)
print(classification_report(reals, preds))